In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
from langchain.chains import LLMChain
from langchain.agents.plan_and_execute import LLMChainStepper, LLMPlanner, PlanOutputParser, Plan, PlanAndExecute

In [2]:
model = ChatOpenAI(temperature=0)

In [44]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessage(content="Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should be the final and complete answer to the question."),
    HumanMessagePromptTemplate.from_template("{input}"),
])

In [45]:
llm_chain_1 = LLMChain(llm=model, prompt=prompt_template)

In [46]:
query = "In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?"

In [47]:
import re

In [48]:
class PlanOP(PlanOutputParser):
    
    def parse(self, text):
        return Plan(steps=re.split("\n\d+\. ", text)[1:])

In [49]:
planner = LLMPlanner(llm_chain=llm_chain_1, output_parser=PlanOP())

In [50]:
prompt_template_2 = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("Original task: {input}"),
    SystemMessagePromptTemplate.from_template("Here are the previous steps you've taken: {previous_steps}"),
    HumanMessagePromptTemplate.from_template("Here is the step you are currently asked to do. Please do this step and only this step:\n\n{current_step}"),
])
llm_chain_2 = LLMChain(llm=model, prompt=prompt_template_2)

In [51]:
stepper = LLMChainStepper(llm_chain=llm_chain_2)

In [52]:
plan_and_execute = PlanAndExecute(planner = planner, executer=stepper)

In [53]:
stepper.llm_chain.verbose = True

In [54]:
plan_and_execute({"input": query})

steps=['Calculate the number of students who enrolled in contemporary dance by multiplying 20 by 20%.', 'Subtract the number of students who enrolled in contemporary dance from the total number of students to get the number of students who did not enroll in contemporary dance.', 'Calculate the number of students who enrolled in jazz dance by multiplying the number of students who did not enroll in contemporary dance by 25%.', 'Subtract the number of students who enrolled in contemporary and jazz dance from the total number of students to get the number of students who enrolled in hip-hop dance.', 'Calculate the percentage of students who enrolled in hip-hop dance by dividing the number of students who enrolled in hip-hop dance by the total number of students and multiplying by 100. This is the final answer.']


> Entering new LLMChain chain...
Prompt after formatting:
System: Original task: In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enroll

{'input': 'In a dance class of 20 students, 20% enrolled in contemporary dance, 25% of the remaining enrolled in jazz dance, and the rest enrolled in hip-hop dance. What percentage of the entire students enrolled in hip-hop dance?',
 'answer': 'Sure! \n\nThe number of students who enrolled in hip-hop dance is 12. \n\nThe total number of students is 20. \n\nSo, the percentage of students who enrolled in hip-hop dance is:\n\n12/20 x 100% = 60%\n\nTherefore, 60% of the entire students enrolled in hip-hop dance.'}